In [2]:
!pip install simpletransformers

In [3]:
# get gpu specs
!nvidia-smi
# Tesla K80  = bad
# Tesla T4   = good
# Tesla P100 = good
# Tesla V100 = very good

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [4]:
!pip install emoji

In [5]:
# imports
import csv
import os
import numpy as np
import pandas as pd
import random
import sys
import warnings
import datetime
import unicodedata
import requests
import math
import re
import logging
import collections
from collections import defaultdict
from pprint import pprint
import emoji


# Dummy metrics: 
import sklearn.metrics
from sklearn.metrics import confusion_matrix

import tensorflow as tf
# BERT/RobBERT
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# muting some output
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')
logging.basicConfig(level=logging.WARNING)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
# metrics: confusion matrix and classification report (with right labels)
def confusion(y_true, y_pred):
    unique_label = np.unique([y_true, y_pred])
    return pd.DataFrame(
        sklearn.metrics.confusion_matrix(y_true, y_pred, labels=unique_label), 
        index=['{:}'.format(LABELS[int(x)]) for x in unique_label], # true rows
        columns=['{:}'.format(LABELS[int(x)]) for x in unique_label] # pred cols
    )

def classification_report(y_true, y_pred):
    return sklearn.metrics.classification_report(y_true, y_pred, digits=4, target_names=LABELS, labels=list(range(len(LABELS))))

## Preprocessing Functions:

In [8]:
def clean_samples(tweet_message):
    
    tweet_message = re.sub(r'(@\w+)','MENTION', tweet_message)
    tweet_message = re.sub(r'(https\S+)','URL', tweet_message)
    tweet_message = re.sub(r'[0-9]+', 'NUMBER', tweet_message)
    tweet_message = emoji.demojize(tweet_message)
    tweet_message = re.sub(r'#', '', tweet_message)
    tweet_message = re.sub(r'[(#.,\/?!@$%^&*)]', '', tweet_message)
    return tweet_message

# Loading Data

In [10]:
path = '' # ADAPT PATH

# manually curate split
train = path + 'DALC2_train_full.csv'
dev = path + 'DALC2_dev_full.csv'
test = path + 'DALC2_test_full.csv'

# Select the task and the number of labels

In [11]:
#LABELS = ["INDIVIDUAL", "GROUP", "OTHER", "NOT"]
#def load(fn):
#    # load data - final
#    full_data = []
#
#    with open(fn, encoding='utf-8') as f:
#        reader = csv.DictReader(f, delimiter='\t')
#        next(reader)
#
#        for index, line in enumerate(reader):
#            #print(line)
#            if line.get('offensive_aggregated', 'NOT').strip() != 'NOT':
#              if line.get('target_aggregated', 'NOT').strip() != 'NOT':
#                  full_data.append([
#                                    preprocess(line['text'], target_pipeline), 
#                                    LABELS.index(line['target_aggregated'].strip())
#                                    ])
#
#    df_full = pd.DataFrame(full_data)
#    #print(df_full.head())
#    df_full.columns = ["text", "labels"]
#    return df_full

#train_df = load('train_final.csv')
#dev_df = load('dev_final.csv')
#test_df = load('test_final.csv')
#train_df = load(train)
#dev_df = load(dev)
#test_df = load(test)

#df_all = pd.concat([train_df, dev_df, test_df])

In [12]:
#LABELS = ["OFFENSIVE", "NOT"]
#
#def load_bin(fn):
#    # load data - final
#   full_data = []
#
#    with open(fn, encoding='utf-8') as f:
#        reader = csv.DictReader(f, delimiter='\t')
#        #print(LABELS)
#        for index, line in enumerate(reader):
#            if line.get('offensive_aggregated', '').strip():
#                if line['offensive_aggregated'] == "IMPLICIT" or line["offensive_aggregated"] == "EXPLICIT":
#                    line["offensive_aggregated"] = "OFFENSIVE"
##            print(clean_samples(line['text']))              
#                full_data.append([
#                                  #preprocess(line['text'], default_pipeline), 
#                                  clean_samples(line['text']),
#                                  LABELS.index(line['offensive_aggregated'].strip()),
#                                  ])
#
#    df_full = pd.DataFrame(full_data)
#    df_full.columns = ["text", "labels"]
#    
#    return df_full

#train_df = load_bin('train_final.csv')
#dev_df = load_bin('dev_final.csv')
#test_df = load_bin('test_final.csv')

#train_df = load_bin(train)
#dev_df = load_bin(dev)
#test_df = load_bin(test)

#df_all = pd.concat([train_df, dev_df, test_df])

In [13]:
# alternative: explicitness

#LABELS = ["EXPLICIT", "IMPLICIT", "NOT"]
#def load_expl(fn):
#    # load data - final
#    full_data = []
#
#    with open(fn, encoding='utf-8') as f:
#        reader = csv.DictReader(f, delimiter='\t')
#
#        for index, line in enumerate(reader):
 #           if line.get('offensive_aggregated', '').strip():
 #               full_data.append([
 #                                 preprocess(line['text'], default_pipeline), 
 #                                 LABELS.index(line['offensive_aggregated'].strip()),
 #                                 ])
#
#    df_full = pd.DataFrame(full_data)
#    df_full.columns = ["text", "labels"]
#    return df_full
#
#train_df = load_expl('train_final.csv')
#dev_df = load_expl('dev_final.csv')
#test_df = load_expl('test_final.csv')
#train_df = load_expl(train)
#dev_df = load_expl(dev)
#test_df = load_expl(test)


df_all = pd.concat([train_df, dev_df, test_df])

# Transformers

In [14]:
EPOCHS = 5

BERTje_baseline_config = {
    'name': 'BERTje baseline',
    'model': 'GroNLP/bert-base-dutch-cased',
    'type': 'bert',
    'train_batch_size': 16, # default 
    'learning_rate': 4e-5, # default
}

RobBERT_baseline_config = {
    'name': 'RobBERT baseline',
    'model': 'pdelobelle/robbert-v2-dutch-base',
    'type': 'roberta',
    'train_batch_size': 16, # default
    'learning_rate': 4e-5, # default
}


def train_bert(from_epoch=False, epochs=EPOCHS, config=BERTje_baseline_config):
    
    print('using model', config['name'])
    # Optional model configuration
    model_args = ClassificationArgs(
                                    overwrite_output_dir=True,
                                    do_lower_case=False, # BERTje is cased, so disable lowercase
                                    save_steps=-1, # save storage space
                                    train_batch_size=config['train_batch_size'], # played with 4, 16, default (8)
                                    learning_rate=config['learning_rate'], # played with 6e-5, 5e-5, 2e-5, 1e-5, default (4e-5)
                                    warmup_steps = 2,
                                    optimizer = 'AdamW',
                                    max_seq_length = 123
                                    )

    model_args.num_train_epochs=epochs

    # allow to continue from previous model:
    #model = "/content/outputs" if from_epoch else config['model']

    # Create a ClassificationModel
    model = ClassificationModel(
        config['type'], 
        #model,
        config['model'],
        num_labels=len(LABELS),
        args=model_args,
        use_cuda=False, # much faster to use GPU
    )

    # Train the model
    if epochs:
        model.train_model(train_df)

    return model

In [16]:
def evaluate_bert(model, df=None):
    # run evaluation for BERT

    metrics = {
        'accuracy': sklearn.metrics.accuracy_score,
        'all': classification_report,
        'confusion': confusion,
    }

    if df is None:
        df = dev_df

    result, model_outputs, wrong_predictions = model.eval_model(df, **metrics)

    return result

## Using

In [ ]:
# BERTje
#b_model = train_bert(0, EPOCHS, config=BERTje_baseline_config)

# DEV
#b_eval = evaluate_bert(b_model, dev_df)
#print(b_eval['all'])
#print(b_eval['confusion'])

# TEST:
#b_eval = evaluate_bert(b_model, test_df)
#print(b_eval['all'])
#print(b_eval['confusion'])

# predict on dev for error analysis
#predictions, raw_outputs = b_model.predict(dev_df['text'].tolist()) 
#print(predictions)

#gold_labels = dev_df['labels'].tolist()
#prediction_data_gold = list(zip(dev_df['text'].tolist(),predictions,gold_labels))
#
#outfile = "predictions_off_target.txt"
#output = open(outfile, "a")
#for elem in prediction_data_gold:
#    text, prediction, gold = elem
#    output.writelines(str(text) + "\t" + str(prediction) + "\t" + str(gold) + "\n")
#output.close


### RobBERT

In [22]:
dev_df_robbert = dev_df.copy()
for i, tweet in dev_df_robbert.iterrows():
    # remove newlines for RobBERT
    dev_df_robbert.loc[i, 'text'] = tweet.text.replace('\n', ' ')

test_df_robbert = test_df.copy()
for i, tweet in test_df_robbert.iterrows():
    # remove newlines for RobBERT
    test_df_robbert.loc[i, 'text'] = tweet.text.replace('\n', ' ')

In [23]:
# RobBERT baseline test # skip

b_model = train_bert(0, EPOCHS, config=RobBERT_baseline_config)
# dev:
b_eval = evaluate_bert(b_model, dev_df)
print(b_eval['all'])
print(b_eval['confusion'])

# test:
b_eval = evaluate_bert(b_model, test_df)
print(b_eval['all'])
print(b_eval['confusion'])


using model RobBERT baseline


Downloading:   0%|          | 0.00/446M [00:00<?, ?B/s]

Some weights of the model checkpoint at pdelobelle/robbert-v2-dutch-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.out_proj.weight', 'cl

Downloading:   0%|          | 0.00/715k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/374k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

  0%|          | 0/6817 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/853 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# predict on dev for error analysis
predictions, raw_outputs = b_model.predict(dev_df['text'].tolist()) 
#
gold_labels = dev_df['labels'].tolist()
prediction_data_gold = list(zip(dev_df['text'].tolist(),predictions,gold_labels))

outfile = "predictions_off_binary.txt"
output = open(outfile, "a")
for elem in prediction_data_gold:
    text, prediction, gold = elem
    output.writelines(str(text) + "\t" + str(prediction) + "\t" + str(gold) + "\n")
output.close